# このコードの目的

１）pytorchの学習プロセス（＝パラメータの調整）について確認する。
２）上記の確認結果に基づいて、基本的な調整パラメータであるbatch_size, learning_rate(lr), weight_decay(正則化係数)について挙動を確認する。

# 結論
batch_sizeは1が最もよく、そのbatch集合ごとの学習でパラメータが特徴を捉えやすいようにマネージする必要がある。
batch_sizeを全学習データにしても、簡単な問題でさえ特徴を捉えきれずパラメータは収束しない。

また、learning_rateとweight_decayに従って、パラメータのepochごとの調整挙動が違う。
以下のスクリプトの出力を確認した上で、適切な設定を選ぶべき。


In [20]:
import torch
import numpy as np

In [62]:
# toy data を生成する

x = torch.tensor([
    [1,0,1,1],
    [0,0,1,0],
    [1,0,0,1],
    [0,1,1,1],
    [1,1,1,1],
    [0,1,0,0],
    [0,1,0,1],
    [1,0,0,0],
    [0,0,0,0],
    [0,0,0,1],
], dtype=torch.float32)

## y は明確な正解となるパラメータがあるように定義する。
y = 0 * x[:,0] + 1 * x[:,1] + 2 * x[:,2] + 3 * x[:,3]

print(x)
print(y)

tensor([[1., 0., 1., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 1.],
        [0., 1., 1., 1.],
        [1., 1., 1., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.]])
tensor([5., 2., 3., 6., 6., 1., 4., 0., 0., 3.])


In [22]:
# シンプルなネットワークを定義する
model = torch.nn.Sequential(
            torch.nn.Linear(4,  1, bias=None),
            )

In [37]:
# 学習率(lr)と正則化項(weight_decay)ごとにoptimizerを定義する。
criterion = torch.nn.L1Loss(reduction='sum') # loss関数はL1ロスのbatch合計とする。
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0)
optimizer__weight_decay = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.1)

In [41]:
# パラメータの確認関数を定義しておく。

def print_parameters(model):
    # 'パラメータと微分値を確認する
    print('↓modelのパラメータを出力↓')
    for pram in model.parameters():
        print('pram={}'.format(pram))
        print('grad={}'.format(pram.grad))

print_parameters(model) #gradはまだ計算されていない

↓modelのパラメータを出力↓
pram=Parameter containing:
tensor([[0.2211, 0.1589, 0.3895, 0.8993]], requires_grad=True)
grad=tensor([[0., 0., 0., 0.]])


In [45]:
# optimizerの挙動を確認する。
_x = x[0]
_y = y[0]
loss = criterion(model(_x), _y)

print('# _xと_yとlossの確認')
print(_x)
print(_y)
print(loss)

print('')

print('# loss.backword()でparametorsごとにgradを計算されます')
loss.backward()
print_parameters(model) # この時gradがL1lossの微分定義に従い、±1*_x になることを確認

print('')

print('# optimizer.step() でoptimizerがパラメータを更新します')
print('### 正則化なしのoptimizer ###')
optimizer.step()
print_parameters(model) 
print('     --> SGDなので、pram = pram - lr * pram.grad となる')
print('### 正則化ありのoptimizer ###')
optimizer__weight_decay.step()
print_parameters(model) 
print('     --> SGDなので、pram = pram - (lr * pram.grad + weight_decay * param)  となる')

print('')

print('# optimizer.zero_grad() でgradを0にリセットする')
optimizer.zero_grad()
print_parameters(model)


# _xと_yとlossの確認
tensor([1., 0., 1., 1.])
tensor(2.)
tensor(0.1429, grad_fn=<L1LossBackward>)

# loss.backword()でparametorsごとにgradを計算されます
↓modelのパラメータを出力↓
pram=Parameter containing:
tensor([[0.2125, 0.1542, 0.7739, 0.8706]], requires_grad=True)
grad=tensor([[-0.9785,  0.0156, -2.9420, -0.9121]])

# optimizer.step() でoptimizerがパラメータを更新します
### 正則化なしのoptimizer ###
↓modelのパラメータを出力↓
pram=Parameter containing:
tensor([[0.3104, 0.1526, 1.0681, 0.9618]], requires_grad=True)
grad=tensor([[-0.9785,  0.0156, -2.9420, -0.9121]])
     --> SGDなので、pram = pram - lr * pram.grad となる
### 正則化ありのoptimizer ###
↓modelのパラメータを出力↓
pram=Parameter containing:
tensor([[0.4051, 0.1495, 1.3517, 1.0434]], requires_grad=True)
grad=tensor([[-0.9475,  0.0308, -2.8352, -0.8159]])
     --> SGDなので、pram = pram - (lr * pram.grad + weight_decay * param)  となる

# optimizer.zero_grad() でgradを0にリセットする
↓modelのパラメータを出力↓
pram=Parameter containing:
tensor([[0.4051, 0.1495, 1.3517, 1.0434]], requires_grad=True)
grad=tensor([[0., 0., 0.

In [47]:
# しかし、batch_sizeをフルサイズにすると、以下のようになる。

    
loss = criterion(model(x), y)

print('# xとyとlossの確認')
print(x)
print(y)
print(loss)

print('')

print('# loss.backword()でparametorsごとにgradを計算されます')
loss.backward()
print_parameters(model)
print('     --> gradはbatch全てのエラー合計に応じて計算される')

print('')

print('# optimizer.step() でoptimizerがパラメータを更新します')
print('### 正則化なしのoptimizer ###')
optimizer.step()
print_parameters(model) 
print('     --> パラメータの更新もSGDに基づいて一括変換される')
print('       --> よって、batch.sizeが大きすぎると学習サンプルごとの特徴を捉えきれない可能性がある')


# xとyとlossの確認
tensor([[1., 0., 1., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 1.],
        [0., 1., 0., 1.]])
tensor([2., 2., 0., 1.])
tensor(13.2137, grad_fn=<L1LossBackward>)

# loss.backword()でparametorsごとにgradを計算されます
↓modelのパラメータを出力↓
pram=Parameter containing:
tensor([[0.0051, 0.1495, 0.9517, 0.6434]], requires_grad=True)
grad=tensor([[ 2., -2.,  2.,  0.]])
     --> gradはbatch全てのエラー合計に応じて計算される

# optimizer.step() でoptimizerがパラメータを更新します
### 正則化なしのoptimizer ###
↓modelのパラメータを出力↓
pram=Parameter containing:
tensor([[-0.1949,  0.3495,  0.7517,  0.6434]], requires_grad=True)
grad=tensor([[ 2., -2.,  2.,  0.]])
     --> パラメータの更新もSGDに基づいて一括変換される
       --> よって、batch.sizeが大きすぎると学習サンプルごとの特徴を捉えきれない可能性が高い


In [63]:
print('batch_sizeを最大で学習する')
for t in range(1000):
    loss = criterion(model(x), y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print_parameters(model)
print('  ----> 正解のパラメータである[0,1,2,3]に収束できない')

batch_sizeを最大で学習する
↓modelのパラメータを出力↓
pram=Parameter containing:
tensor([[ 0.1051,  0.1495, -0.1483, -1.2567]], requires_grad=True)
grad=tensor([[32., 32., 32., 52.]])
  ----> 正解のパラメータである[0,1,2,3]に収束できない


In [64]:
print('batch_size=1で学習する')
import random
for t in range(1000):
    batch_index = random.choice(range(x.shape[0]))
    loss = criterion(model(x[batch_index]), y[batch_index])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print_parameters(model)
print('  ----> 正解のパラメータである[0,1,2,3]に収束できた')

batch_size=1で学習する
↓modelのパラメータを出力↓
pram=Parameter containing:
tensor([[-0.0949,  0.8495,  2.0517,  2.9433]], requires_grad=True)
grad=tensor([[1., 1., 1., 1.]])
  ----> 正解のパラメータである[0,1,2,3]に収束できない
